<h1><b>Statistique en Bioinformatique : </b> TME2 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> objectif 1: comprendre les différences d'alignement global et local, </li>
<li> objectif 2: reconstruire une matrice de substitution du type Blosum, </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME2.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre sur moodle </li>
</ul>
</div>

Nom etudiant 1 : Antoine Grislain
<br>
Nom etudiant 2 : Simon Liétar
<br>

<b> Exercice 1 </b>: On va étudier un alignement “difficile” entre la protéine emph50s ribosomal L20 chez A. aeolicus, 
et la protéine ligase UBR5 chez l’homme (L’alignement structural est montré ci-dessous). 
<br>

<img src="bacthum.png" alt="Smiley face" height="210" width="202"> 



<br><br>
<b>A.</b> Récupérez la séquence de la protéine “E3 ubiquitin ligase UBR5” chez l’homme et de la protéine “50S ribosomal L20” chez A. aeolicus 
via le site <a href="http://www.uniprot.org/">Uniprot</a>  au format .fasta.

<br>
<b>B.</b> A l’aide du site <a href="https://www.ebi.ac.uk/Tools/psa/">psa</a>, alignez ces séquences. Commentez les faibles pourcentage d’identité malgré le bon alignement des ces structures.


Réponse : 

| Alignement | Longueur | Identité        | Similarité      | Gaps               | Score |
|------------|----------|-----------------|-----------------|--------------------|-------|
| global     | 2826     | 29/2826 (1.0 %) | 51/2826 (1.8 %) | 2735/2826 (96.8 %) | 28.5  |
| local      | 35       | 13/35 (37.1 %)  | 20/35 (57.1 %)  | 1/35 (2.9 %)       | 49.0  |

Les deux séquences ne sont pas du tout de la même taille donc l'identité est mauvaise.

<b>C.</b> Récupérez la séquence de la protéine “metL Bifunctional aspartokinase/homoserine dehydrogenase 2” et de la protéine “lysC Lysine-sensitive aspartokinase 3” chez E. coli via le site <a href="http://www.uniprot.org/"> Uniprot </a> au format .fasta. Reproduisez, à l’aide du site <a href="https://www.ebi.ac.uk/Tools/psa/"> psa</a>, l’alignement global et local vu en TD. Observez la modification des résultats en changeant la matrice de substitution et/ou les pénalités de gap (“Gap_penalty” et “Extend_penalty”).

Avec les valeurs par défaut (gap penalty = 10 et extend penalty = 0.5), on a les résultats suivants :

| Alignement | Longueur | Identité         | Similarité       | Gaps             | Score |
|------------|----------|------------------|------------------|------------------|-------|
| global     | 854      | 136/854 (15.9 %) | 209/854 (24.5 %) | 449/854 (52.6 %) | 351.0 |
| local      | 482      | 133/482 (27.6 %) | 205/482 (42.5 %) | 85/482 (17.6 %)  | 353.5 |

En baissant les coûts (gap penalty = 5 et extend penalty = 0.2), l'alignement local est quasiment identique à l'alignement global car les gaps sont beaucoups moins coûteux dans l'alignement donc ils sont plus nombreux. Le score des deux alignements augmente nettement.

<b>Exercice 2</b>: Score alignement global, local <br>
Faire une fonction pour calculer : 1) la pourcentage de identité, 2) la pourcentage de similarité, 3) le score d’un alignement en utilisant la matrix blosum62. Le calcul de la similarité doit prendre en considération les acides amine ayant la même propriété physico chimique. Dans notre cas, tous les acides amine qui ont des valeurs supérieur à zéro dans la matrice BLOSUM62. Le score basé sur la matrice blosum doit aussi prendre en considération les deux pénalités de gaps, ouverture et extension. Teste vos fonctions en utilisant le fichier test.fasta


In [1]:
import numpy as np

#variable initiation
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'X']

#parameters
q = 21
gap_open = -5
gap_ext = 0.5

#files
input_test_f = 'test.fasta'
input_blosum_f = 'BLOSUM62.txt'

#For a simple test use:
input_test_f = 'testToy.fasta'


In [2]:
#Read the test.fasta file
def read_fasta (input_f):
	"""
	Input1 input_f: fasta file name
	Output1 arr_seqs: List of sequences
	REMARK : We keep only the sequences, identifiers are not necessary
	"""

	with open(input_f, 'r') as in_f:
		return [line.strip() for line in in_f if line[0] != '>']

testAln = read_fasta(input_test_f)
print (testAln)
#It shlould print ['ACCD', 'A-CD'] when using toy

['ACCD', 'A-CD']


In [3]:
#read Blosum
def read_blosum(input_f: str):
	"""
	Input1 input_f : blosum file name
	Output1: a matrix of dimention [q, q]
	"""

	with open(input_f, 'r') as in_f:
		lines = in_f.read().replace('  ', ' ').splitlines()
		data = [line[1:].strip() for line in lines[7:]]
		alphabet = lines[6].strip().split(' ')

	indices = [alphabet.index(a) for a in aa]
	matrix = np.loadtxt(data, dtype=int)
	matrix = matrix[:, indices][indices, :]

	assert matrix.shape[0] == matrix.shape[1] == q
	return matrix

matr_62 = read_blosum(input_blosum_f)
print(matr_62)


[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0  0]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3 -1]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2 -1]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3 -1]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -1]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -1]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2 -1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -1]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -1]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2]
 [ 1 -1  1

In [4]:
def indentite_calcul(seq1: str, seq2: str, gap_open: float, gap_ext: float, matr_62: np.ndarray):
	"""
	Input1 seq1 : sequence
	Input2 seq2 : sequence
	Input3 gap_open : gap open penality
	Input4 gap_ext : gap extention penality
	Input5 matr_62 :  a matrix of dimention [q-1, q-1] representing substitution matrix
	Output1 ident: sequence identity
	Output2 similaire: sequence similarity
	Output3 score: aligment score
	"""

	score = 0.0
	similarity = 0
	ident = 0

	gap = False

	for a1, a2 in zip(seq1, seq2):
		if (a1 == '-') or (a2 == '-'):
			if gap:
				score -= gap_ext
			else:
				score += gap_open
				gap = True
		else:
			aa_score = matr_62[aa.index(a1), aa.index(a2)]
			gap = False
			score += aa_score

			if a1 == a2:
				ident += 1
			if aa_score > 0.0:
				similarity += 1

	return (
		ident / len(seq1),
		similarity / len(seq1),
		score
	)

identitee,similarity,score = indentite_calcul(testAln[0], testAln[1], gap_open, gap_ext, matr_62)
print ("identitee= ", identitee , " similarity= ", similarity, " score= ", score )

#identitee=  0.75  similarity=  0.75  score=  24.0

identitee=  0.75  similarity=  0.75  score=  14.0


<b>Exercice 3</b>: Matrice de substitution <br>
Faire une programme (plusieurs fonctions seront nécessaires) pour produire une matrix de substitution comme Blosum. Utiliser l’alignement du fichiers <b>blocks.dat</b> Vous pouvez tester vous fonctions sue le fichier <b>blockToy.dat</b>. 


In [5]:
from itertools import combinations, combinations_with_replacement
from collections import Counter

input_block_f = 'blockToy.dat'


#For a simple test do:
q = 3
aa = ['A', 'B', 'C']


In [6]:
#read alignment file
from io import StringIO
import sys

def readAlnFile(input_f: str):
	"""
	Input1 input_f : Alignment File name
	Output1 aln : List of aligned sequences
	Output2 cols : Number of colums (L)
	"""

	with open(input_f) as file:
		lines = file.read().splitlines()
		return lines, len(lines[0])

aln, cols  = readAlnFile(input_block_f)
#cols = 43
#cols = 5 testing toy
print(aln)
print(cols)


['ABCCB', 'ACABA', 'CBCCB', 'ACBBB']
5


In [7]:
#compute fij frequences
import math

def frequencies(aln: list[str], cols: int, aa: list[str]):
	"""
	Input1 aln : List of aligned sequences
	Input2 cols : Number of colums in the aligment (L)
	Input3 aa: residue alphabet
	Output1  pairs_freq_dict: a dictionary containing residue pair frequencies
	"""

	matrix = np.zeros((q, q), dtype=int)

	for pos in range(cols):
		occurences = np.zeros(q, dtype=int)

		for seq in aln:
			occurences[aa.index(seq[pos])] += 1

		matrix_pos = occurences[:, None] * occurences
		matrix_pos[*np.diag_indices(q, 2)] = [math.comb(a, 2) for a in occurences]

		assert np.tril(matrix_pos).sum() == math.factorial(q)

		matrix += matrix_pos

	# On retourne une matrice symétrique plutôt qu'un dictionnaire pour faciliter les calculs suivants.
	return matrix

pairs_freq_dict = frequencies(aln, cols, aa)
print (pairs_freq_dict)
#{('A', 'A'): 3, ('A', 'B'): 4, ('A', 'C'): 5, ('B', 'B'): 5, ('B', 'C'): 10, ('C', 'C'): 3}

[[ 3  4  5]
 [ 4  5 10]
 [ 5 10  3]]


In [8]:
#compute T
def calcule_T(pairs_freq_dict: np.ndarray):
	"""
	Input1 pairs_freq_dict: a dictionary containing residue pair frequencies
	  Output1 : T, the overall sum
	"""

	return np.tril(pairs_freq_dict).sum()

T = calcule_T(pairs_freq_dict)
print (T)
#30

30


In [9]:
#compute pi
def calcule_p_is(pairs_freq_dict: np.ndarray, T: int):
	"""
	Compute the probability of each residue.
	Input1 pairs_freq_dict: a dictionary containing residue pair frequencies
	Input2 T : the overall sum
	Output1 pi: List of probabilities
	REMARK : You can compute qij directly from pairs_freq_dict or create a function to do so.
	"""

	return (0.5 * (1.0 + np.eye(len(pairs_freq_dict))) * pairs_freq_dict / T).sum(axis=0)

pi = calcule_p_is(pairs_freq_dict, T)
print (pi)
#[0.25, 0.4, 0.35]

[0.25 0.4  0.35]


In [10]:
#compute Sij
def calcule_S_i_j(pairs_freq_dict: np.ndarray, p: np.ndarray, T: float):
	"""
	Input1 pairs_freq_dict: a dictionary containing residue pair frequencies
	Input2 p: List of probabilities for each residue
	Input3 T: the overall sum
	Output1 s: Score for each residue pair
	"""

	return np.log(pairs_freq_dict / T / p[:, None] / p[None, :] / (2.0 - np.eye(len(pairs_freq_dict))))

Sij = calcule_S_i_j(pairs_freq_dict, pi, T)
print (Sij)
#[[0.47000362924573563, -0.40546510810816444, -0.048790164169431945], [-0.40546510810816444, 0.04082199452025478, 0.17435338714477774], [-0.048790164169431945, 0.17435338714477774, -0.20294084399669016]]

[[ 0.47000363 -0.40546511 -0.04879016]
 [-0.40546511  0.04082199  0.17435339]
 [-0.04879016  0.17435339 -0.20294084]]


In [11]:
#print Matrix
Sij_c = [["{:0.2f}".format(x) for x in l] for l in Sij]
print('Blosum matrix from Blocks database:')
row_format ="{:>5}|" * (len(aa) + 1)
print(row_format.format("", *aa))
for aa_, row in zip(aa, Sij_c):
	print(row_format.format(aa_, *row))

Blosum matrix from Blocks database:
     |    A|    B|    C|
    A| 0.47|-0.41|-0.05|
    B|-0.41| 0.04| 0.17|
    C|-0.05| 0.17|-0.20|
